In [ ]:
import json
import os
import re
import spacy
import urllib.parse
import unicodedata

data_folder = os.path.join(os.getcwd(), 'data')
extracted_data = []

# Loading the spacy model
nlp = spacy.load('en_core_web_sm')


# a list of culinary measurement words and cooking adjectives and descriptions to exclude from results
measurement_words = {
    'acidic', 'add', 'aged', 'airy', 'alternative', 'aromatic', 'astringent', 'bake', 'bag', 'bags', 'barrel', 
    'battered', 'beat', 'beaten', 'big', 'bit', 'bite', 'bitesize', 'bland', 'blanched', 'blend', 'blended', 'blackened', 
    'bitter', 'block', 'boil', 'boneless', 'box', 'boxes', 'braise', 'braised', 'bread', 
    'breaded', 'brewed', 'brined', 'broil', 'brush', 'burn', 'burnt', 'buttery', 'bunch', 'bunches', 'can', 'candied', 
    'cap', 'caps', 'caramelize', 'carve', 'carved', 'carton', 'casserole', 'center', 'center-cut', 'centercut', 'chalky', 
    'charred', 'chewy', 'choice', 'chill', 'chop', 'chopped', 'chunk', 'chunks', 'chunky', 'classico', 'clove', 'cloves', 'cloying', 
    'coat', 'coated', 'cold', 'combine', 'conimex', 'cool', 'cook', 'cooked', 'cover', 
    'creamy', 'cracked', 'crisp', 'crispy', 'crunchy', 'crumbly', 'crush', 'crushed', 'cube', 'cubed', 'cubes', 'cup',
    'cured', 'cut', 'dash', 'dashes', 'deglaze', 'dehydrated', 'deep-fry', 'deep-fried', 'degrees', 'diameter', 'dice', 
    'diced', 'directions', 'dip', 'dipped', 'dissolve', 'dollop', 'dozen', 'drain', 'dram', 'drizzle', 'dried', 
    'drizzled', 'dunk', 'dunked', 'dust', 'effervescent', 'ferment', 'fermented', 'filter', 'filled', 'finely', 'unfiltered',
    'flambé', 'flaky', 'flat', 'flattened', 'flavored', 'flip', 'fold', 'foamy', 'fry', 'frying', 
    'frothy', 'firm', 'fluid', 'foamy', 'frozen', 'fried', 'g', 'gallon', 'gallons', 'garnish', 'garnished', 'gelatinous',
    'glaze', 'glog', 'glug', 'gob', 'golden', 'gooey', 'gram', 'grams', 'grate', 
    'grated', 'grating', 'grill', 'grind', 'gritty', 'ground', 'head', 'heads', 'heat', 'hellmann', 'hellmanns', 
    'herbal', 'hole', 'holes', 'home', 'hour', 'hours', 'hot', 'hunk', 'inch', 'inches', 'inch-diameter', 
    'inch-thick', 'infuse', 'infused', 'instant', 'jar', 'jars', 'jigger', 'jug', 'julienne', 'jumbo', 'juicy', 
    'kg', 'kilogram', 'kilograms', 'knead', 'kneaded', 'layer', 'layered', 'layers', 'lb', 'lbs', 'leaf', 'leaves', 
    'lengthways', 'light', 'little', 'liter', 'liters', 'lukewarm', 'luke', 'mash', 'marinate', 'marinated', 
    'medley', 'medium', 'metallic', 'microwave', 'milky', 'milligram', 'milligrams', 'milliliter', 'milliliters', 
    'mince', 'minced', 'minute', 'minutes', 'minim', 'mix', 'mixed', 'ml', 'molded', 'moist', 'mushy', 'nip', 
    'nutty', 'one', 'ones', 'optional', 'ounce', 'ounces', 'oz', 'oz.', 'package', 'packages', 'pan', 'pans', 
    'parboiled', 'part', 'parts', 'pat', 'peck', 'peeled', 'peel', 'peels', 'peppery', 'perishable', 'piece', 
    'pieces', 'pickled', 'pinch', 'pint', 'pints', 'pkg', 'pkgs', 'plate', 'plates', 'poach', 'poached', 'pony', 'pour', 
    'pounded', 'powdered', 'preference', 'preserved', 'puree', 'quarter', 'quarters', 'quart', 'quarts', 'pinches',
    'raw', 'refrigerate', 'refrigerated', 'rich', 'ripe', 'ripened', 'roast', 'roasted', 'roll', 'room', 'paperthin', 'paper', 'thin',
    'salted', 'salt spoon', 'saute', 'sachet', 'scald', 'scalded', 'scrubbed', 'scoop', 'scoops', 'season', 'seasons', 'serve', 
    'sheet', 'sheets', 'shredded', 'shot', 'silky', 'simmer', 'size', 'sliced', 'slice', 'slices', 'sliver', 'toaster', 'l', 'colours',
    'small', 'smidgen', 'smoke', 'smoky', 'smooth', 'soggy', 'sous-vide', 'splash', 'spongy', 'spoonful', 'spray', 'pouch', 'uncle', 'bens',
    'sprinkled', 'stalk', 'stalks', 'steamed', 'steep', 'steeped', 'stem', 'stems', 'stew', 'stick', 'sticks', 'strain', 'garnishes', 'gr', 'm', 'e', 'glace', 'glaces', 'demiglace',
    'strained', 'stuff', 'stuffed', 'sugar-free', 'sweetener', 'sweat', 'sweet', 't', 'tad', 'tablespoon', 'tablespoons', 'breyers', 'ith', 'cd',
    'tablespoonful', 'tbsp', 'tbs', 'teacup', 'teaspoon', 'teaspoons', 'teaspoonful', 'teaspooon', 'temperature', 'tender', 'tablespoonfuls',
    'third', 'thirds', 'thick', 'thickness', 'tiny', 'toast', 'toasted', 'toss', 'total', 'tps', 'tube', 'tub', 'tsps', 'tbsps', 'rabe', 'approx', 'approximately', 'approximatelyi',
    'uncooked', 'unctuous', 'unsalted', 'use', 'velvety', 'warm', 'washed', 'waxy', 'well', 'whip', 'whipped', 'testing', 'purposes', 'gmofree',
    'whisked', 'whole', 'wilted', 'woven', 'work', 'wrapped', 'zesty', 'heart', 'hearts', 'type', 'types', 'cage', 'parchment', 'microplane', 'grater',
    'strip', 'pound', 'pounds', 'taste', 'cups', 'food', 'seedless', 'bonein', 'skinless', 'crosswise', 'lengthwise', 'length', 'inchwide', 'superfine',
    'super','fine', 'handful', 'ingredient', 'squeeze', 'tspa', 'couple', 'handfuls', 'palmfuls', 'palmful', 'style', 'sift', 'spectrum', 'homemade',
    'sprig', 'sprigs', 'root', 'roots', 'powder', 'paste', 'cooking', 'skinon', 'extract', 'round', 'bar', 'peeler', 'boiling', 'sweet', 'semisweet', 'allpurpose',
    'all-purpose', 'pod', 'litre', 'tspoon', 'tbspoon', 'slit', 'mccormick', 'perfect', 'option', 'options', 'el', 'tl', 'ingredients', 'store', 'stores', 'places', 'place',
    'boil', 'boilinbag', 'half', 'halves', 'container', 'bottle', 'and', 'ready', 'direction', 'saltfree', 'hawaiianstyle', 'lesssodium', 'large', 'tbso',
    'fatfree', 'gm', 'tsp', 'tbsp', 'pc', 'sp', 'c', 'grocery', 'list', 'tblsp', 'bertolli', 'f', 'evoo', 'dish', 'dishes', 'thread', 'threads', 'skillet', 'skillets',
    'whisk', 'min', 'max', 'leftover', 'leftovers', 'content', 'contents', 'cm', 'wedge', 'wedges', 'plu', 'brand', 'storebought', 'coin', 'coins', 'tolerance', 'kitchen', 'tips',
    'skin-on', 'goodquality', 'crusty', 'mediumhot', 'hot', 'containers', 'surface', 'hands', 'glass', 'glasses', 'substitute', 'substitutes', 'round', 'rounds', 'groceries', 
    'brushing', 'bowl', 'bowls', 'hand', 'cans', 'flavor', 'flavour', 'flavors', 'flavours', 'recipe', 'recipes', 'combination', 'cms', 'gms', 'pcs', 'minimum', 'maximum',
    'storage', 'situations', 'purpose', 'all', 'strips', 'loaf', 'loaves', 'square', 'circle', 'triangle',  'squares', 'circles', 'triangles', 'see', 'lengths', 'ball', 'balls', 'summer',
    'wear', 'gloves', 'eyes', 'kids', 'husband', 'remove', 'gekookte', 'gepelde', 'knorr', 'posje', 'pakje', 'pak', 'pos', 'good', 'thereabouts', 'matchsticks', 'i', 'dairy', 'nondairy', 'instructions',
    'prepared', 'precut', 'kikkoman', 'toppings', 'topping', 'franks', 'martins', 'marthas', 'bestquality', 'firmripe', 'firm', 'ripe', 'fresh',
    'freshground', 'smokepoint', 'lowsodium', 'islands', 'cholesterol', 'low', 'cholestrol_free', 'market', 'foods', 'widthwise', 'necks', 'bulb',
    'confectioners', 'grade', 'a', 'b', 'dayold','daysold','dayolds', 'ends', 'savory', 'tidbits', 'coarseground', 'fineground', 'w', 'chef', 'paul', 'serving', 'servings'
}


# extract noun and proper-noun ingredient words using POS and exclude stop words
def extract_ingredients(ingredient_description):
    doc = nlp(ingredient_description)
    ingredients = [token.text for token in doc if token.pos_ in {'NOUN', 'PROPN'} and not token.is_stop]
    return ' '.join(ingredients)

# normalise ingredient words
def normalize_ingredient(ingredient):
    ingredient = ingredient.lower().strip()  # lower cases and strip leading/trailing spaces
    ingredient = re.sub(r'[^\w\s]', '', ingredient)  #removing punctuation
    ingredient = re.sub(r'\s+', ' ', ingredient)  # removing extra whitespaces
    ingredient = re.sub(r'\d+\/\d+|\d+|¼|½|¾|⅓|⅔|⅛|⅜|⅝|⅞', '', ingredient)  # removing fractions and numbers
    ingredient = re.sub(r'[^\x00-\x7F]+', '', ingredient)  # Remove non-ASCII characters
    ingredient_words = ingredient.split()
    ingredient = ' '.join(word for word in ingredient.split() if word not in measurement_words)
    #ingredient = '_'.join(word for word in ingredient.split() if word not in measurement_words and not nlp.vocab[word].is_stop)
    return ingredient

def preprocess_ingredients(ingredients):
    normalized_ingredients = [normalize_ingredient(ingredient) for ingredient in ingredients]
    extracted_ingredients = [extract_ingredients(ingredient) for ingredient in normalized_ingredients]
    return extracted_ingredients

for filename in os.listdir(data_folder):
    if filename.endswith('.json'):
        with open(os.path.join(data_folder, filename), 'r') as file:
            data = json.load(file)
            for item in data:
                recipe_name = item.get('name')
                ingredients = item.get('ingredients')
                cuisine = item.get('cuisine')
                if ingredients:
                    ingredients = preprocess_ingredients(ingredients)
                extracted_data.append({'recipe_name': recipe_name, 'ingredients': ingredients, 'cuisine': cuisine})

print(extracted_data[:10])




In [ ]:
# a translation table for unwanted characters to be replaced/removed
single_char_replacements = str.maketrans({
    ' ': '_', '&': 'and', '/': '_', '"': '', "'": '', '’': '', '|': '', '(': '', ')': '', ',': '', '<':'', '>':'', '»':'', '«':'', '=':'',
    '{': '', '}': '', '‘': '', '[': '', ']': '', '#': '', '–': '', '+': '', '®': '', ':': '', '“': '', '♥':'', '☀':'', '`':'', '©':'', 'ø': 'o',
    '”': '', '�': '', '!': '', '—': '_', '$':'', '@':'_AT_', '?':'', 'Đ':'D', 'ı':'i', ';':'', '%':'perc', '*':'', '\\':'', 'ß':'ss',
    '\u2028': '', '\ufeff': '', '\u0096': '', '\u200e': '', '\u200f': '', '\u0097': '', '\u200b':'', '\u2155':'', '\u00ef': '', '\u00bf': '', '\u00bd': ''
})

In [ ]:
import json
import os
import re
import spacy
import urllib.parse
import unicodedata

def normalize_text(text):
    # normalise unicode characters and remove diacritics
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    #translation table to remove unwanted characters
    text = text.translate(single_char_replacements)
    # unwanted multi-characters 
    text = text.replace('œu', 'eu').replace('21⁄2', 'two_and_half').replace('__ˊॢo◡ुoˋॢ_⁎','').replace('1⁄5', 'one_fifth').replace('Ai1⁄2oli','Aioli').replace('nbsp','').replace('...','').replace('.','').replace('~','').replace('__','_').replace('-','_')
    # Remove non latin letters languages including Japanese, Chinese, Korean, and Thai characters
    text = re.sub(r'[\u3000-\u303F\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF\uF900-\uFAFF\uAC00-\uD7AF\u1100-\u11FF\u3130-\u318F\u0E00-\u0E7F\u0370-\u03FF\u0980-\u09FF\u1200-\u137F]', '', text)
    return text

In [ ]:
import json
import os
import re
import spacy
import urllib.parse
import unicodedata
from rdflib import Graph, URIRef, Literal, Namespace, RDF, RDFS

def create_valid_iri(base, name):
    # normalisation and URL encoding
    name = normalize_text(name)
    return f'{base}{urllib.parse.quote(name)}'

with open('recipes.ttl', 'w', encoding='utf-8') as f:
    f.write('@prefix ex: <http://example.org/ontology/> .\n')
    f.write('@prefix recipe: <http://example.org/recipe/> .\n')
    f.write('@prefix ingredient: <http://example.org/ingredient/> .\n')
    f.write('@prefix cuisine: <http://example.org/cuisine/> .\n')
    f.write('@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n')
    f.write('@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n\n')

    # classes
    f.write('ex:Recipe a rdfs:Class .\n')
    f.write('ex:Ingredient a rdfs:Class .\n')
    f.write('ex:Cuisine a rdfs:Class .\n\n')

    # properties (and their domain and range)
    f.write('ex:RecipeHasIngredient a rdf:Property ;\n')
    f.write('    rdfs:domain ex:Recipe ;\n')
    f.write('    rdfs:range ex:Ingredient .\n\n')

    f.write('ex:RecipeBelongsToCuisine a rdf:Property ;\n')
    f.write('    rdfs:domain ex:Recipe ;\n')
    f.write('    rdfs:range ex:Cuisine .\n\n')

    f.write('ex:IngredientBelongstoRecipe a rdf:Property ;\n')
    f.write('    rdfs:domain ex:Ingredient ;\n')
    f.write('    rdfs:range ex:Recipe .\n\n')

    f.write('ex:CuisineHasRecipe a rdf:Property ;\n')
    f.write('    rdfs:domain ex:Cuisine ;\n')
    f.write('    rdfs:range ex:Recipe .\n\n')

    f.write('ex:RecipeHasName a rdf:Property ;\n')
    f.write('    rdfs:domain ex:Recipe ;\n')
    f.write('    rdfs:range rdfs:Literal .\n\n')

    for recipe in extracted_data:
        recipe_name = normalize_text(recipe["recipe_name"])
        cuisine_name = normalize_text(recipe["cuisine"])
        recipe_uri = create_valid_iri('recipe:', recipe_name)
        cuisine_uri = create_valid_iri('cuisine:', cuisine_name)

        f.write(f'{recipe_uri} a ex:Recipe .\n')
        f.write(f'{cuisine_uri} a ex:Cuisine .\n')
        f.write(f'{recipe_uri} ex:RecipeBelongsToCuisine {cuisine_uri} .\n')
        
        recipe_name_literal = normalize_text(recipe["recipe_name"]).replace('"', '(').replace('"', ')').replace("'", '').replace("’", "")
        f.write(f'{recipe_uri} ex:RecipeHasName "{recipe_name_literal}" .\n')

        for ingredient in recipe['ingredients']:
            ingredient_name = normalize_ingredient(ingredient).replace("'", '').replace("’", "")
            if ingredient_name:
                ingredient_uri = create_valid_iri('ingredient:', ingredient_name)
                f.write(f'{ingredient_uri} a ex:Ingredient .\n')
                f.write(f'{recipe_uri} ex:RecipeHasIngredient {ingredient_uri} .\n')
                f.write(f'{ingredient_uri} ex:IngredientBelongstoRecipe {recipe_uri} .\n')
        
        f.write(f'{cuisine_uri} ex:CuisineHasRecipe {recipe_uri} .\n')

print("RDF data written to recipes.ttl")

Queries

In [ ]:
from rdflib import Graph
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns

g = Graph()
g.parse("recipes.ttl", format="turtle")

# SPARQL query to get all ingredients and usage counts per cuisine
query = """
PREFIX ex: <http://example.org/ontology/>
PREFIX recipe: <http://example.org/recipe/>
PREFIX ingredient: <http://example.org/ingredient/>
PREFIX cuisine: <http://example.org/cuisine/>

SELECT ?cuisine ?ingredient (COUNT(?recipe) AS ?usageCount)
WHERE {
  ?recipe ex:RecipeHasIngredient ?ingredient .
  ?recipe ex:RecipeBelongsToCuisine ?cuisine .
}
GROUP BY ?cuisine ?ingredient
ORDER BY ?cuisine DESC(?usageCount)
"""
results = g.query(query)

# results conversion
data = []
for row in results:
    cuisine = str(row.cuisine).split('/')[-1]
    ingredient = str(row.ingredient).split('/')[-1]
    usageCount = int(row.usageCount)
    data.append((cuisine, ingredient, usageCount))
df = pd.DataFrame(data, columns=["cuisine", "ingredient", "usageCount"])

#top 10 ingredients per cuisine
top_ingredients_per_cuisine = df.groupby("cuisine").apply(lambda x: x.nlargest(10, "usageCount")).reset_index(drop=True)

#Print in table format
print(tabulate(top_ingredients_per_cuisine, headers='keys', tablefmt='psql'))

#visualisation
plt.figure(figsize=(14, 10))
sns.barplot(data=top_ingredients_per_cuisine, x='usageCount', y='ingredient', hue='cuisine', dodge=False)
plt.title('Top 10 Ingredients per Cuisine')
plt.xlabel('Usage Count')
plt.ylabel('Ingredient')
plt.legend(title='Cuisine', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()

In [ ]:
from rdflib import Graph
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

g = Graph()
g.parse("recipes.ttl", format="turtle")

# SPARQL query to count recipes per cuisine
query = """
PREFIX ex: <http://example.org/ontology/>
PREFIX recipe: <http://example.org/recipe/>
PREFIX cuisine: <http://example.org/cuisine/>

SELECT ?cuisine (COUNT(?recipe) AS ?recipeCount)
WHERE {
  ?recipe ex:RecipeBelongsToCuisine ?cuisine .
}
GROUP BY ?cuisine
ORDER BY DESC(?recipeCount)
"""
results = g.query(query)

data = []
for row in results:
    cuisine = str(row.cuisine).split('/')[-1]
    recipeCount = int(row.recipeCount)
    data.append((cuisine, recipeCount))
df = pd.DataFrame(data, columns=["cuisine", "recipeCount"])
print(df)

# Visualisation
plt.figure(figsize=(12, 8))
sns.barplot(data=df, x='cuisine', y='recipeCount', palette='viridis')
plt.title('Number of Recipes per Cuisine')
plt.xlabel('Cuisine')
plt.ylabel('Number of Recipes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Map

In [ ]:
from rdflib import Graph
import pandas as pd
import requests
from dash import Dash, dcc, html
import plotly.express as px
from dash.dependencies import Input, Output
import re
import time

g = Graph()
g.parse("recipes.ttl", format="turtle")

# internal SPARQL query to get all ingredients with usage counts per cuisine
query = """
PREFIX ex: <http://example.org/ontology/>
PREFIX recipe: <http://example.org/recipe/>
PREFIX ingredient: <http://example.org/ingredient/>
PREFIX cuisine: <http://example.org/cuisine/>

SELECT ?cuisine ?ingredient (COUNT(?recipe) AS ?usageCount)
WHERE {
  ?recipe ex:RecipeHasIngredient ?ingredient .
  ?recipe ex:RecipeBelongsToCuisine ?cuisine .
}
GROUP BY ?cuisine ?ingredient
ORDER BY ?cuisine DESC(?usageCount)
"""
results = g.query(query)

data = []
for row in results:
    cuisine = str(row.cuisine).split('/')[-1]
    ingredient = str(row.ingredient).split('/')[-1]
    usageCount = int(row.usageCount)
    data.append((cuisine, ingredient, usageCount))
df = pd.DataFrame(data, columns=["cuisine", "ingredient", "usageCount"])

# extract the base ingredient name using first word
def get_base_ingredient(ingredient_name):
    return re.split(r'_|\s', ingredient_name)[0]
df['base_ingredient'] = df['ingredient'].apply(get_base_ingredient)

# aggregate the usage counts by cuisine and base ingredient
aggregated_df = df.groupby(['cuisine', 'base_ingredient']).agg({'usageCount': 'sum'}).reset_index()

# Function and external SPARQL query to get country name and coordinates from Wikidata
def get_country_coordinates(cuisine_name):
    if cuisine_name == 'American':
        # manually add coordinates for American cuisine (United States cuisine) 
        return 'United States', 37.0902, -95.7129
    else:
        cuisine_name += ' cuisine'
        sparql_country = f"""
        SELECT ?countryLabel WHERE {{
          ?cuisine rdfs:label "{cuisine_name}"@en .
          ?cuisine wdt:P17 ?country .
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        """  
    url = 'https://query.wikidata.org/sparql'
    
    for attempt in range(3):
        try:
            response_country = requests.get(url, params={'query': sparql_country, 'format': 'json'})
            response_country.raise_for_status()
            response_country = response_country.json()
            
            try:
                country = response_country['results']['bindings'][0]['countryLabel']['value']
            except IndexError:
                return None, None, None
            
            sparql_coordinates = f"""
            SELECT ?coordinates WHERE {{
              ?country rdfs:label "{country}"@en ;
                       wdt:P31  wd:Q6256;
                       wdt:P625 ?coordinates .
            }}
            """
            response_coordinates = requests.get(url, params={'query': sparql_coordinates, 'format': 'json'})
            response_coordinates.raise_for_status()
            response_coordinates = response_coordinates.json()
            
            try:
                coordinates = response_coordinates['results']['bindings'][0]['coordinates']['value']
                lon, lat = coordinates.replace('Point(', '').replace(')', '').split(' ')
                return country, float(lat), float(lon)
            except IndexError:
                return country, None, None
        
        except requests.exceptions.RequestException:
            time.sleep(2)
    
    return None, None, None

# coordinates for each cuisine
cuisine_data = []
for cuisine in aggregated_df['cuisine'].unique():
    country, lat, lon = get_country_coordinates(cuisine)
    if country and lat and lon:
        cuisine_data.append({'cuisine': cuisine, 'country': country, 'lat': lat, 'lon': lon})

cuisine_df = pd.DataFrame(cuisine_data)
aggregated_df = aggregated_df.merge(cuisine_df, on='cuisine')

# using Dash app to create map. circle size and colour depend on usage count of ingredient and cuisine respectively.
app = Dash(__name__)

app.layout = html.Div([
    dcc.Input(id='ingredient-input', value='', type='text', placeholder='Enter ingredient'),
    dcc.Graph(id='map')
])

@app.callback(
    Output('map', 'figure'),
    [Input('ingredient-input', 'value')]
)
def update_map(input_value):
    if input_value:
        filtered_df = aggregated_df[aggregated_df['base_ingredient'].str.contains(input_value, case=False)]
    else:
        filtered_df = aggregated_df
    
    fig = px.scatter_mapbox(
        filtered_df,
        lat="lat",
        lon="lon",
        hover_name="cuisine",
        hover_data=["base_ingredient", "usageCount"],
        color="cuisine",
        size="usageCount",
        size_max=15,
        zoom=1,
        height=600
    )
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)